In [91]:
import openai
import json
import re
import pandas as pd
import requests
import time
from google.colab import userdata

df = pd.read_csv("/content/all_data.csv")

df = df[
    (df['action_type'].isna() == False) &
    (df['self-typical-confusion'].isna() == False) &
    (df['self-typical-interactions'].isna() == False) &
    (df['self-typical-confusion'] > 2) &
    (df['self-typical-interactions'] > 2)  &
    (df['state'].isna() == False)
].reset_index(drop=True)

In [92]:
df

,Unnamed: 0,qid,scenario,question,ground_truth,student_incorrect_solution,student_profile,teacher_described_confusion,self-correctness,self-typical-confusion,self-typical-interactions,conversation,state,action_type,action_text,reward,next_state,next_action_type,next_action_text,done
0,0,5000012,1,Nancy is filling an aquarium for her fish. She...,First calculate the volume of the aquarium by ...,The aquarium has a volume of 4 x 6 x 3 = 72 cu...,Steven is a 7th grade student. He has difficul...,He added a step after completing the problem.,Yes,3.0,3.0,"Teacher: (probing)Steven, If you had 4 of some...",START,probing,"Teacher: (probing)Steven, If you had 4 of some...",NaN,Steven: I would have 12 of something.,probing,Teacher: (probing)So if Nancy triples the 18 c...,0
1,1,5000012,1,Nancy is filling an aquarium for her fish. She...,First calculate the volume of the aquarium by ...,The aquarium has a volume of 4 x 6 x 3 = 72 cu...,Steven is a 7th grade student. He has difficul...,He added a step after completing the problem.,Yes,3.0,3.0,"Teacher: (probing)Steven, If you had 4 of some...",Steven: I would have 12 of something.,probing,Teacher: (probing)So if Nancy triples the 18 c...,NaN,Steven: She would have 54 cubic feet of water.,generic,Teacher: (generic)Exactly correct!,0
2,2,5000012,1,Nancy is filling an aquarium for her fish. She...,First calculate the volume of the aquarium by ...,The aquarium has a volume of 4 x 6 x 3 = 72 cu...,Steven is a 7th grade student. He has difficul...,He added a step after completing the problem.,Yes,3.0,3.0,"Teacher: (probing)Steven, If you had 4 of some...",Steven: She would have 54 cubic feet of water.,generic,Teacher: (generic)Exactly correct!,NaN,NaN,NaN,NaN,1
3,14,5000982,3,James has 20 pairs of red socks and half as ma...,He has 20/2=10 black socks\nSo he has 20+10=30...,James has 20/2 = 10 pairs of black socks.\nJam...,DeAndre is a 7th grade student. He has problem...,He was technically correct.,Yes,5.0,5.0,"Teacher: (probing)DeAndre, you successfully an...",START,probing,"Teacher: (probing)DeAndre, you successfully an...",NaN,Student: James had 10 pairs of black socks.,probing,Teacher: (probing)could you tell me the total ...,0
4,15,5000982,3,James has 20 pairs of red socks and half as ma...,He has 20/2=10 black socks\nSo he has 20+10=30...,James has 20/2 = 10 pairs of black socks.\nJam...,DeAndre is a 7th grade student. He has problem...,He was technically correct.,Yes,5.0,5.0,"Teacher: (probing)DeAndre, you successfully an...",Student: James had 10 pairs of black socks.,probing,Teacher: (probing)could you tell me the total ...,NaN,Student: James had 20 pairs of red socks.,probing,Teacher: (probing)could you tell me the total ...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12013,20134,5000818,2,A restaurant buffet has 36 different dishes av...,There are fresh mangos in 36 / 6 = 6 dishes.\n...,There are 3 + 36/6 + 1 = 10 dishes with mango ...,Winnie is a 7th grade student. She struggle mo...,Repeated subtraction by deducting the number o...,Yes,3.0,3.0,"Teacher: (generic)Hi, could you please walk me...",Student: Sure. I started by counting the numbe...,focus,Teacher: (focus)How many total dishes with man...,NaN,NaN,focus,Teacher: (focus),0
12014,20138,5000818,2,A restaurant buffet has 36 different dishes av...,There are fresh mangos in 36 / 6 = 6 dishes.\n...,There are 3 + 36/6 + 1 = 10 dishes with mango ...,Winnie is a 7th grade student. She struggle mo...,Repeated subtraction by deducting the number o...,Yes,3.0,3.0,"Teacher: (generic)Hi, could you please walk me...",Student: There are 10 dishes with mangoes in t...,focus,Teacher: (focus)Oliver can pick the mangoes ou...,NaN,Student: Oliver won't eat 2 of the dishes with...,focus,Teacher: (focus)Do you see you are subtracting...,0
12015,20139,5000818,2,A restaurant buffet has 36 different dishes av...,There are fresh mangos in 36 / 6 = 6 dishes.\n...,There are 3 + 36/6 + 1 = 10 dishes with mango

In [93]:
columns = [
    'question',
    'qid',
    'scenario',
    'ground_truth',
    'student_incorrect_solution',
    'student_profile',
    'conversation',
    'state',
    'action_text',
    'action_type',
    'done',
    'self-typical-confusion',
    'self-typical-interactions'
]

new_df = df[columns]

state_df = new_df.copy()
state_df['full_state'] = None

for i in range(0, len(new_df)):
  if 'START' in str(new_df.iloc[i]['state']):
    state_df.at[i, 'full_state'] = ['', 'START']
    continue

  prev_action_text = new_df.iloc[i-1]['action_text']
  curr_state = new_df.iloc[i]['state']

  state_df.at[i, 'full_state'] = [prev_action_text, curr_state]

state_df

state_df['conversation']
curr_pos = 1
if len(state_df) > 0:
  state_df.loc[0, 'convo_turn'] = curr_pos

for i in range(1, len(state_df)):
    # Check if a new conversation starts (0 follows a 1)
    if state_df.loc[i-1, 'done'] == 1 and state_df.loc[i, 'done'] == 0:
        # Reset position counter for new conversation
        curr_pos = 1
    else:
        # Increment position within the same conversation
        curr_pos += 1

    # Assign the current position within conversation
    state_df.loc[i, 'convo_turn'] = curr_pos * 1

state_df['convo_turn'] = state_df['convo_turn'].astype(int)
state_df['old_state'] = state_df['state']
state_df['state'] = state_df['full_state']
state_df['next_action'] = state_df['action_text']

state_df
columns = [
    'question',
    'qid',
    'scenario',
    'ground_truth',
    'state',
    'next_action',
    'convo_turn',
    'action_type',
    'done'
    ]

state_df = state_df[columns]

In [94]:
state_df

,question,qid,scenario,ground_truth,state,next_action,convo_turn,action_type,done
0,Nancy is filling an aquarium for her fish. She...,5000012,1,First calculate the volume of the aquarium by ...,"[, START]","Teacher: (probing)Steven, If you had 4 of some...",1,probing,0
1,Nancy is filling an aquarium for her fish. She...,5000012,1,First calculate the volume of the aquarium by ...,"[Teacher: (probing)Steven, If you had 4 of som...",Teacher: (probing)So if Nancy triples the 18 c...,2,probing,0
2,Nancy is filling an aquarium for her fish. She...,5000012,1,First calculate the volume of the aquarium by ...,[Teacher: (probing)So if Nancy triples the 18 ...,Teacher: (generic)Exactly correct!,3,generic,1
3,James has 20 pairs of red socks and half as ma...,5000982,3,He has 20/2=10 black socks\nSo he has 20+10=30...,"[, START]","Teacher: (probing)DeAndre, you successfully an...",1,probing,0
4,James has 20 pairs of red socks and half as ma...,5000982,3,He has 20/2=10 black socks\nSo he has 20+10=30...,"[Teacher: (probing)DeAndre, you successfully a...",Teacher: (probing)could you tell me the total ...,2,probing,0
...,...,...,...,...,...,...,...,...,...
12013,A restaurant buffet has 36 different dishes av...,5000818,2,There are fresh mangos in 36 / 6 = 6 dishes.\n...,"[Teacher: (generic)Hi, could you please walk m...",Teacher: (focus)How many total dishes with man...,25,focus,0
12014,A restaurant buffet has 36 different dishes av...,5000818,2,There are fresh mangos in 36 / 6 = 6 dishes.\n...,[Teacher: (focus)How many total dishes with ma...,Teacher: (focus)Oliver can pick the mangoes ou...,26,focus,0
12015,A restaurant buffet has 36 different dishes av...,5000818,2,There are fresh mangos in 36 / 6 = 6 dishes.\n...,[Teacher: (focus)Oliver can pick the mangoes o...,Teacher: (focus)Do you see you are subtracting...,27,focus,0
12016,A restaurant buffet has 36 different dishes av...,5000818,2,There are fresh mangos in 36 / 6 = 6 dishes.\n...,[Teacher: (focus)Do you see you are subtractin...,Teacher: (focus)But you already got 4 dishes b...,28,focus,0


In [95]:
count = 0
for idx in state_df.loc[state_df['next_action'].isna()].index:
    # print(state_df.at[idx, 'state'])
    # print(state_df.at[idx, 'qid'])
    state_df.drop(index=idx, inplace=True)
    count = count + 1
print(count)

state_df.reset_index(drop=True, inplace=True)


0


In [96]:
# Method 1: Check for NaN in specific column
nan_count = df["state"].isna().sum()
print(f"Number of NaN values in 'state' column: {nan_count}")

# Method 2: Check for NaN across the entire DataFrame
total_nan = df.isna().sum().sum()
print(f"Total NaN values in entire DataFrame: {total_nan}")

# Method 3: Get a count of NaN values per column
nan_by_column = df.isna().sum()
print("NaN count by column:")
print(nan_by_column)

# Method 4: Find which rows have NaN in a specific column
rows_with_nan = df[df["state"].isna()]
print(f"Rows with NaN in 'state' column:")
print(rows_with_nan)

# Method 5: Check if a specific row/cell has NaN
if pd.isna(df.loc[130, "state"]):
    print("Row 130, column 'state' contains NaN")

Number of NaN values in 'state' column: 0
Total NaN values in entire DataFrame: 17303
NaN count by column:
Unnamed: 0                         0
qid                                0
scenario                           0
question                           0
ground_truth                       0
student_incorrect_solution         0
student_profile                    0
teacher_described_confusion        0
self-correctness                   0
self-typical-confusion             0
self-typical-interactions          0
conversation                       0
state                              0
action_type                        0
action_text                        0
reward                         12018
next_state                      1329
next_action_type                1978
next_action_text                1978
done                               0
dtype: int64
Rows with NaN in 'state' column:
Empty DataFrame
Columns: [Unnamed: 0, qid, scenario, question, ground_truth, student_incorrect_solution, st

In [97]:
state_df

,question,qid,scenario,ground_truth,state,next_action,convo_turn,action_type,done
0,Nancy is filling an aquarium for her fish. She...,5000012,1,First calculate the volume of the aquarium by ...,"[, START]","Teacher: (probing)Steven, If you had 4 of some...",1,probing,0
1,Nancy is filling an aquarium for her fish. She...,5000012,1,First calculate the volume of the aquarium by ...,"[Teacher: (probing)Steven, If you had 4 of som...",Teacher: (probing)So if Nancy triples the 18 c...,2,probing,0
2,Nancy is filling an aquarium for her fish. She...,5000012,1,First calculate the volume of the aquarium by ...,[Teacher: (probing)So if Nancy triples the 18 ...,Teacher: (generic)Exactly correct!,3,generic,1
3,James has 20 pairs of red socks and half as ma...,5000982,3,He has 20/2=10 black socks\nSo he has 20+10=30...,"[, START]","Teacher: (probing)DeAndre, you successfully an...",1,probing,0
4,James has 20 pairs of red socks and half as ma...,5000982,3,He has 20/2=10 black socks\nSo he has 20+10=30...,"[Teacher: (probing)DeAndre, you successfully a...",Teacher: (probing)could you tell me the total ...,2,probing,0
...,...,...,...,...,...,...,...,...,...
12013,A restaurant buffet has 36 different dishes av...,5000818,2,There are fresh mangos in 36 / 6 = 6 dishes.\n...,"[Teacher: (generic)Hi, could you please walk m...",Teacher: (focus)How many total dishes with man...,25,focus,0
12014,A restaurant buffet has 36 different dishes av...,5000818,2,There are fresh mangos in 36 / 6 = 6 dishes.\n...,[Teacher: (focus)How many total dishes with ma...,Teacher: (focus)Oliver can pick the mangoes ou...,26,focus,0
12015,A restaurant buffet has 36 different dishes av...,5000818,2,There are fresh mangos in 36 / 6 = 6 dishes.\n...,[Teacher: (focus)Oliver can pick the mangoes o...,Teacher: (focus)Do you see you are subtracting...,27,focus,0
12016,A restaurant buffet has 36 different dishes av...,5000818,2,There are fresh mangos in 36 / 6 = 6 dishes.\n...,[Teacher: (focus)Do you see you are subtractin...,Teacher: (focus)But you already got 4 dishes b...,28,focus,0


In [98]:
import json, re, time, os, requests, pandas as pd
from pathlib import Path

# ---------- helper -----------------------------------------------------------
def extract_json(text):
    try:
        data = json.loads(text)
        if validate_schema(data):
            return data
    except json.JSONDecodeError:
        pass
    json_pattern = r'\s*(\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\})\s*'
    for potential_json in reversed(re.findall(json_pattern, text, re.DOTALL)):
        try:
            data = json.loads(potential_json)
            if validate_schema(data):
                return data
        except json.JSONDecodeError:
            continue
    start_idx = text.rfind('{')
    if start_idx != -1:
        brace_count, i = 1, start_idx + 1
        while i < len(text) and brace_count:
            brace_count += (text[i] == '{') - (text[i] == '}')
            i += 1
        if not brace_count:
            try:
                data = json.loads(text[start_idx:i])
                if validate_schema(data):
                    return data
            except json.JSONDecodeError:
                pass
    pattern = r'\{\s*"state":\s*\{[^}]*\},\s*"action":\s*\{[^}]*\}\s*\}'
    for potential_json in re.findall(pattern, text, re.DOTALL):
        try:
            data = json.loads(potential_json.strip())
            if validate_schema(data):
                return data
        except json.JSONDecodeError:
            continue
    return None

def validate_schema(data):
    if not isinstance(data, dict) or {"state", "action"} - data.keys():
        return False
    state, action = data["state"], data["action"]
    for k in ("listen_to_feedback", "problem_progress", "correct_solution"):
        if not isinstance(state.get(k), int):
            return False
    if state["listen_to_feedback"] not in (0, 1) or \
       not 0 <= state["problem_progress"] <= 100 or \
       state["correct_solution"] not in (0, 1):
        return False
    if not isinstance(action.get("next_action_hint_strength"), int) or \
       not 0 <= action["next_action_hint_strength"] <= 4:
        return False
    cat = str(action.get("next_action_category", "")).lower().strip()
    sub = str(action.get("next_action_subcategory", "")).lower().strip()
    allowed = {
        "focus": {"seek next step", "confirm calculation", "re-direct to sub-problem", "highlight missing info"},
        "probing": {"ask for explanation", "seek self-correction", "hypothetical variation", "check understanding/concept", "encourage comparison"},
        "telling": {"partial reveal (strategy)", "full reveal (answer)", "corrective explanation"},
        "generic": {"greeting/farewell", "acknowledgment/praise", "summarize progress", "general inquiry/filler"},
    }
    return cat in allowed and sub in allowed[cat]

# ---------- configuration ----------------------------------------------------
checkpoint = "part1_in_progress.csv"
output_file = "part1_mathdial_complete.csv"
# source_file = "initial_state.csv"   # change to your real source file
api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY missing")

# ---------- load dataframe ---------------------------------------------------
if Path(checkpoint).exists():
    state_df = pd.read_csv(checkpoint)
    print(f"Loaded checkpoint with {len(state_df)} rows")
else:
    print("Using same state_df as before!")

for col in ["raw_model_response", "listen_to_feedback", "problem_progress",
            "correct_solution", "next_action_hint_strength",
            "next_action_category", "next_action_subcategory",
            "extraction_failed", "retry_count", "error_message"]:
    if col not in state_df.columns:
        state_df[col] = pd.NA

# ---------- main loop --------------------------------------------------------
requests_per_minute = 20
delay_between_requests = 60 / requests_per_minute

todo_df = state_df[state_df["raw_model_response"].isna()].copy()
total_remaining = len(todo_df)
print(f"{total_remaining} rows need processing")

for counter, (idx, row) in enumerate(todo_df.iterrows(), 1):
    if idx not in state_df.index:
        continue
    if counter > 1:
        time.sleep(delay_between_requests)

    print(f"[{counter}/{total_remaining}] Row {idx}: sending request")

    question, correct_solution = row.get("question", ""), row.get("ground_truth", "")
    student_profile = row.get("student_profile", "")
    current_state = row.get("state", "")
    action_text = row.get("action_text", "")
    action_type = str(row.get("action_type", "")).lower()
    is_start = isinstance(current_state, str) and "START" in current_state

    # Add this as context
    prompt = f"""
You are analyzing math tutoring sessions to extract features for an AI tutor.

CONTEXT:
- Question: {question}
- Correct solution: {correct_solution}
- Student profile: {student_profile}
- Current conversation (teacher feedback, student response): {current_state}
- Next action text: {action_text}
- Next action type: {action_type}
- Is start of conversation: {is_start}

TASK:
Extract the following features from this tutoring interaction:

1. LISTEN TO FEEDBACK:
   - 1 if the student has incorporated the teacher's feedback in the current conversation state
   - 0 if not

2. PROBLEM PROGRESS - Numerical estimate (0-100) of how close, given the student's current conversation state, the student is to the correct solution. For example:
  - 0: No progress/completely wrong approach/conversation just starting
  - 25: Beginning to understand but major errors remain
  - 50: Halfway to solution with some key insights
  - 75: Minor errors but mostly correct approach
  - 100: Complete and correct solution

  IMPORTANT: Carefully analyze the current conversation state to determine progress.
  At the start of a conversation with no student responses yet, progress should be 0.
  (You may use any value between 0-100, not just the examples above)

3. CORRECT SOLUTION:
   - 0 if the student's final answer is incorrect
   - 1 if correct

4. NEXT ACTION HINT STRENGTH (0-4):
   How strong is the teacher's hint for the next action? (0=very minimal hint, 4=essentially giving away the entire solution)

5. NEXT ACTION SUBTYPE:
    Choose exactly one of the following, depending on the next action type:

    If action type is "focus":
    - "Seek Next Step"
    - "Confirm Calculation"
    - "Re-direct to Sub-Problem"
    - "Highlight Missing Info"

    If action type is "probing":
    - "Ask for Explanation"
    - "Seek Self-Correction"
    - "Hypothetical Variation"
    - "Check Understanding/Concept"
    - "Encourage Comparison"

    If action type is "telling":
    - "Partial Reveal (Strategy)"
    - "Full Reveal (Answer)"
    - "Corrective Explanation"

    If action type is "generic":
    - "Greeting/Farewell"
    - "Acknowledgment/Praise"
    - "Summarize Progress"
    - "General Inquiry/Filler"

FORMAT YOUR RESPONSE AS JSON:
{{
  "state": {{
    "listen_to_feedback": 0 or 1,
    "problem_progress": integer_between_0_and_100,
    "correct_solution": 0 or 1,
    "reasoning": briefly_explain_why_student_is_or_isnt_correct
  }},
  "action": {{
    "next_action_hint_strength": integer_between_0_and_4,
    "next_action_category": "{action_type}",
    "next_action_subcategory": "pick_the_exact_one_subcategory_name"
  }}
}}

IMPORTANT:
- Return ONLY the JSON object with no additional explanation
- All numeric fields must be valid integers
""".strip()

    payload = {
        "model": "gpt-4.1-mini-2025-04-14",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 400
    }
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}

    retry, max_retries = 0, 3
    while retry < max_retries:
        try:
            resp = requests.post("https://api.openai.com/v1/chat/completions",
                                 headers=headers, json=payload)
            if resp.status_code == 429:
                print("Rate‑limit hit, sleeping 60 s")
                time.sleep(60)
                continue
            resp.raise_for_status()
            out = resp.json()["choices"][0]["message"]["content"]
            state_df.at[idx, "raw_model_response"] = out
            data = extract_json(out)
            if not data:
                raise ValueError("schema fail")
            s, a = data["state"], data["action"]
            state_df.at[idx, "listen_to_feedback"] = s["listen_to_feedback"]
            progress = 0 if is_start and s["problem_progress"] > 25 else s["problem_progress"]
            state_df.at[idx, "problem_progress"] = progress
            state_df.at[idx, "correct_solution"] = s["correct_solution"]
            state_df.at[idx, "next_action_hint_strength"] = a["next_action_hint_strength"]
            state_df.at[idx, "next_action_category"] = a["next_action_category"]
            state_df.at[idx, "next_action_subcategory"] = a["next_action_subcategory"]
            print(f"Row {idx} processed successfully")
            break
        except Exception as e:
            retry += 1
            print(f"Row {idx} error ({retry}/{max_retries}): {e}")
            if retry == max_retries:
                state_df.at[idx, "extraction_failed"] = True
                state_df.at[idx, "error_message"] = str(e)
            else:
                time.sleep(2)

    tmp = checkpoint + ".tmp"
    state_df.to_csv(tmp, index=False)
    os.replace(tmp, checkpoint)
    print(f"Checkpoint saved after row {idx}")

# ---------- final save -------------------------------------------------------
state_df.to_csv(output_file, index=False)
print("Run complete — final file written")


Streaming output truncated to the last 5000 lines.
[10364/12018] Row 10363: sending request
Row 10363 processed successfully
Checkpoint saved after row 10363
[10365/12018] Row 10364: sending request
Row 10364 processed successfully
Checkpoint saved after row 10364
[10366/12018] Row 10365: sending request
Row 10365 processed successfully
Checkpoint saved after row 10365
[10367/12018] Row 10366: sending request
Row 10366 processed successfully
Checkpoint saved after row 10366
[10368/12018] Row 10367: sending request
Row 10367 processed successfully
Checkpoint saved after row 10367
[10369/12018] Row 10368: sending request
Row 10368 processed successfully
Checkpoint saved after row 10368
[10370/12018] Row 10369: sending request
Row 10369 processed successfully
Checkpoint saved after row 10369
[10371/12018] Row 10370: sending request
Row 10370 processed successfully
Checkpoint saved after row 10370
[10372/12018] Row 10371: sending request
Row 10371 processed successfully
Checkpoint saved af

In [99]:
import pandas as pd

# Predefined mapping dictionary (1-based hint strengths).
ACTION_MAP = {
  "focus": {
    "Seek Next Step": {
      "1": 0,
      "2": 1,
      "3": 2,
      "4": 3,
      "5": 4
    },
    "Confirm Calculation": {
      "1": 5,
      "2": 6,
      "3": 7,
      "4": 8,
      "5": 9
    },
    "Re-direct to Sub-Problem": {
      "1": 10,
      "2": 11,
      "3": 12,
      "4": 13,
      "5": 14
    },
    "Highlight Missing Info": {
      "1": 15,
      "2": 16,
      "3": 17,
      "4": 18,
      "5": 19
    }
  },
  "probing": {
    "Ask for Explanation": {
      "1": 20,
      "2": 21,
      "3": 22,
      "4": 23,
      "5": 24
    },
    "Seek Self-Correction": {
      "1": 25,
      "2": 26,
      "3": 27,
      "4": 28,
      "5": 29
    },
    "Hypothetical Variation": {
      "1": 30,
      "2": 31,
      "3": 32,
      "4": 33,
      "5": 34
    },
    "Check Understanding/Concept": {
      "1": 35,
      "2": 36,
      "3": 37,
      "4": 38,
      "5": 39
    },
    "Encourage Comparison": {
      "1": 40,
      "2": 41,
      "3": 42,
      "4": 43,
      "5": 44
    }
  },
  "telling": {
    "Partial Reveal (Strategy)": {
      "1": 45,
      "2": 46,
      "3": 47,
      "4": 48,
      "5": 49
    },
    "Full Reveal (Answer)": {
      "1": 50,
      "2": 51,
      "3": 52,
      "4": 53,
      "5": 54
    },
    "Corrective Explanation": {
      "1": 55,
      "2": 56,
      "3": 57,
      "4": 58,
      "5": 59
    }
  },
  "generic": {
    "Greeting/Farewell": {
      "1": 60,
      "2": 61,
      "3": 62,
      "4": 63,
      "5": 64
    },
    "Acknowledgment/Praise": {
      "1": 65,
      "2": 66,
      "3": 67,
      "4": 68,
      "5": 69
    },
    "Summarize Progress": {
      "1": 70,
      "2": 71,
      "3": 72,
      "4": 73,
      "5": 74
    },
    "General Inquiry/Filler": {
      "1": 75,
      "2": 76,
      "3": 77,
      "4": 78,
      "5": 79
    }
  }
}

def get_next_action_id(row):
    """
    row["next_action_category"] -> e.g. "Focus"
    row["next_action_subcategory"] -> e.g. "Seek Next Step"
    row["next_action_hint_strength"] -> now 0..4 in CSV,
                                        but dictionary expects 1..5 as string
    """
    cat = row["next_action_category"]
    sub = row["next_action_subcategory"]
    # We add 1 because your CSV hint_strength is 0..4; the dictionary keys are 1..5 as strings
    hint_str = str(row["next_action_hint_strength"] + 1)
    return ACTION_MAP[cat][sub][hint_str]

# Load the CSV
df = pd.read_csv("/content/part1_mathdial_complete.csv")

In [100]:
df

,question,qid,scenario,ground_truth,state,next_action,convo_turn,action_type,done,raw_model_response,listen_to_feedback,problem_progress,correct_solution,next_action_hint_strength,next_action_category,next_action_subcategory,extraction_failed,retry_count,error_message
0,Nancy is filling an aquarium for her fish. She...,5000012,1,First calculate the volume of the aquarium by ...,"['', 'START']","Teacher: (probing)Steven, If you had 4 of some...",1,probing,0,"```json\n{\n ""state"": {\n ""listen_to_feedb...",0.0,0.0,0.0,1.0,probing,Ask for Explanation,NaN,NaN,NaN
1,Nancy is filling an aquarium for her fish. She...,5000012,1,First calculate the volume of the aquarium by ...,"['Teacher: (probing)Steven, If you had 4 of so...",Teacher: (probing)So if Nancy triples the 18 c...,2,probing,0,"{\n ""state"": {\n ""listen_to_feedback"": 1,\...",1.0,25.0,0.0,1.0,probing,Check Understanding/Concept,NaN,NaN,NaN
2,Nancy is filling an aquarium for her fish. She...,5000012,1,First calculate the volume of the aquarium by ...,['Teacher: (probing)So if Nancy triples the 18...,Teacher: (generic)Exactly correct!,3,generic,1,"{\n ""state"": {\n ""listen_to_feedback"": 1,\...",1.0,100.0,1.0,1.0,generic,Acknowledgment/Praise,NaN,NaN,NaN
3,James has 20 pairs of red socks and half as ma...,5000982,3,He has 20/2=10 black socks\nSo he has 20+10=30...,"['', 'START']","Teacher: (probing)DeAndre, you successfully an...",1,probing,0,"```json\n{\n ""state"": {\n ""listen_to_feedb...",0.0,0.0,0.0,1.0,probing,Ask for Explanation,NaN,NaN,NaN
4,James has 20 pairs of red socks and half as ma...,5000982,3,He has 20/2=10 black socks\nSo he has 20+10=30...,"['Teacher: (probing)DeAndre, you successfully ...",Teacher: (probing)could you tell me the total ...,2,probing,0,"{\n ""state"": {\n ""listen_to_feedback"": 1,\...",1.0,75.0,0.0,2.0,probing,Check Understanding/Concept,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12013,A restaurant buffet has 36 different dishes av...,5000818,2,There are fresh mangos in 36 / 6 = 6 dishes.\n...,"['Teacher: (generic)Hi, could you please walk ...",Teacher: (focus)How many total dishes with man...,25,focus,0,"```json\n{\n ""state"": {\n ""listen_to_feedb...",0.0,50.0,0.0,2.0,focus,Seek Next Step,NaN,NaN,NaN
12014,A restaurant buffet has 36 different dishes av...,5000818,2,There are fresh mangos in 36 / 6 = 6 dishes.\n...,['Teacher: (focus)How many total dishes with m...,Teacher: (focus)Oliver can pick the mangoes ou...,26,focus,0,"```json\n{\n ""state"": {\n ""listen_to_feedb...",1.0,50.0,0.0,2.0,focus,Seek Next Step,NaN,NaN,NaN
12015,A restaurant buffet has 36 different dishes av...,5000818,2,There are fresh mangos in 36 / 6 = 6 dishes.\n...,"[""Teacher: (focus)Oliver can pick the mangoes ...",Teacher: (focus)Do you see you are subtracting...,27,focus,0,"```json\n{\n ""state"": {\n ""listen_to_feedb...",0.0,40.0,0.0,3.0,focus,Re-direct to Sub-Problem,NaN,NaN,NaN
12016,A restaurant buffet has 36 different dishes av...,5000818,2,There are fresh mangos in 36 / 6 = 6 dishes.\n...,['Teacher: (focus)Do you see you are subtracti...,Teacher: (focus)But you already got 4 dishes b...,28,focus,0,"```json\n{\n ""state"": {\n ""listen_to_feedb...",1.0,75.0,0.0,2.0,focus,Seek Next Step,NaN,NaN,NaN


In [101]:
# Method 1: Check for NaN in specific column
nan_count = df["state"].isna().sum()
print(f"Number of NaN values in 'state' column: {nan_count}")

# Method 2: Check for NaN across the entire DataFrame
total_nan = df.isna().sum().sum()
print(f"Total NaN values in entire DataFrame: {total_nan}")

# Method 3: Get a count of NaN values per column
nan_by_column = df.isna().sum()
print("NaN count by column:")
print(nan_by_column)

# Method 4: Find which rows have NaN in a specific column
rows_with_nan = df[df["state"].isna()]
print(f"Rows with NaN in 'state' column:")
print(rows_with_nan)

# Method 5: Check if a specific row/cell has NaN
if pd.isna(df.loc[130, "state"]):
    print("Row 130, column 'state' contains NaN")

Number of NaN values in 'state' column: 0
Total NaN values in entire DataFrame: 36310
NaN count by column:
question                         0
qid                              0
scenario                         0
ground_truth                     0
state                            0
next_action                      0
convo_turn                       0
action_type                      0
done                             0
raw_model_response               0
listen_to_feedback              64
problem_progress                64
correct_solution                64
next_action_hint_strength       64
next_action_category            64
next_action_subcategory         64
extraction_failed            11954
retry_count                  12018
error_message                11954
dtype: int64
Rows with NaN in 'state' column:
Empty DataFrame
Columns: [question, qid, scenario, ground_truth, state, next_action, convo_turn, action_type, done, raw_model_response, listen_to_feedback, problem_progress, correct_

In [103]:
count = 0
for idx in state_df.loc[state_df['extraction_failed'] == 1].index:
    print(state_df.at[idx, 'state'])
    print(state_df.at[idx, 'qid'])
    # state_df.drop(index=idx, inplace=True)
    count = count + 1
print(count)

# state_df.reset_index(drop=True, inplace=True)


['Teacher: (probing)we know that yesterday Lara collected 80 cans and Sarah collected 50. what is the total?', 'Student: Yesterday, Lara and Sarah collected a total of 80 + 50 = 130 cans.']
5001028
["Teacher: (probing)Good job finding out Jane's age in six years. Dara should be half of that age, but what is half of 34? ", "Student: Half of 34 is 17. So, Dara will be 17 years old in six years. Then I subtracted 6 years from 17 years to find out Dara's current age, which is 11 years old. Finally, I subtracted 11 years from 25 years to find out how many more years Dara needs to wait to reach the minimum age required by the company to be employed, which is 14 years."]
5000512
['Teacher: (focus)OK, you are right up to your last step. If she ran 9 miles in the third week, and 5 miles less in the fourth, how many miles did she run in the fourth week?', 'Student: She ran 4 miles in the fourth week. 9 - 5 = 4.']
5000659
["Teacher: (focus)yes that's right. So how much in the two years?", "Studen

In [2]:
import pandas as pd
import json
import re

# Load the CSV
df = pd.read_csv("/content/part1_mathdial_complete.csv")

# Get rows with extraction failures
failed_rows = df[df["extraction_failed"] == True].copy()
print(f"Found {len(failed_rows)} rows with extraction failures")

# Define the allowed subcategories for each action category
ALLOWED_SUBCATEGORIES = {
    "focus": {"seek next step", "confirm calculation", "re-direct to sub-problem", "highlight missing info"},
    "probing": {"ask for explanation", "seek self-correction", "hypothetical variation", "check understanding/concept", "encourage comparison"},
    "telling": {"partial reveal (strategy)", "full reveal (answer)", "corrective explanation"},
    "generic": {"greeting/farewell", "acknowledgment/praise", "summarize progress", "general inquiry/filler"},
}

# Create a mapping from subcategory to the correct action category
SUBCATEGORY_TO_CATEGORY = {}
for category, subcategories in ALLOWED_SUBCATEGORIES.items():
    for subcategory in subcategories:
        SUBCATEGORY_TO_CATEGORY[subcategory.lower()] = category

# Function to extract JSON from text (similar to the original function)
def extract_json(text):
    try:
        data = json.loads(text)
        return data
    except json.JSONDecodeError:
        pass

    # Try to find JSON patterns
    json_pattern = r'\s*(\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\})\s*'
    for potential_json in reversed(re.findall(json_pattern, text, re.DOTALL)):
        try:
            data = json.loads(potential_json)
            return data
        except json.JSONDecodeError:
            continue

    # Handle code block format (common in LLM outputs)
    code_block_pattern = r'```(?:json)?\s*(\{[\s\S]*?\})\s*```'
    matches = re.findall(code_block_pattern, text, re.DOTALL)
    for match in matches:
        try:
            data = json.loads(match)
            return data
        except json.JSONDecodeError:
            continue

    return None

# Function to fix the JSON with the correct action category
def fix_json(json_data):
    if not json_data or "action" not in json_data:
        return None

    action = json_data["action"]
    subcategory = str(action.get("next_action_subcategory", "")).lower().strip()
    current_category = str(action.get("next_action_category", "")).lower().strip()

    # Check if the subcategory exists in our mapping
    if subcategory in SUBCATEGORY_TO_CATEGORY:
        correct_category = SUBCATEGORY_TO_CATEGORY[subcategory]

        # If the category is incorrect, update it
        if current_category != correct_category:
            action["next_action_category"] = correct_category
            print(f"Fixed: Changed category from '{current_category}' to '{correct_category}' for subcategory '{subcategory}'")

    return json_data

# Process each failed row
fixed_count = 0
for idx, row in failed_rows.iterrows():
    raw_response = row["raw_model_response"]

    # Skip if no raw response
    if pd.isna(raw_response):
        continue

    # Extract the JSON from the raw response
    json_data = extract_json(raw_response)

    if json_data:
        # Fix the JSON
        fixed_json = fix_json(json_data)

        if fixed_json:
            # Update the dataframe with the fixed values
            state, action = fixed_json["state"], fixed_json["action"]

            df.at[idx, "listen_to_feedback"] = state["listen_to_feedback"]
            df.at[idx, "problem_progress"] = state["problem_progress"]
            df.at[idx, "correct_solution"] = state["correct_solution"]
            df.at[idx, "next_action_hint_strength"] = action["next_action_hint_strength"]
            df.at[idx, "next_action_category"] = action["next_action_category"]
            df.at[idx, "next_action_subcategory"] = action["next_action_subcategory"]

            # Mark as fixed
            df.at[idx, "extraction_failed"] = False
            df.at[idx, "error_message"] = "Fixed"
            fixed_count += 1

print(f"Fixed {fixed_count} out of {len(failed_rows)} failed rows")

# Save the updated dataframe
df.to_csv("/content/part1_mathdial_complete_fixed.csv", index=False)
print("Updated CSV saved to: /content/part1_mathdial_complete_fixed.csv")

# Let's also compute the next_action_id using the provided function
ACTION_MAP = {
  "focus": {
    "Seek Next Step": {
      "1": 0,
      "2": 1,
      "3": 2,
      "4": 3,
      "5": 4
    },
    "Confirm Calculation": {
      "1": 5,
      "2": 6,
      "3": 7,
      "4": 8,
      "5": 9
    },
    "Re-direct to Sub-Problem": {
      "1": 10,
      "2": 11,
      "3": 12,
      "4": 13,
      "5": 14
    },
    "Highlight Missing Info": {
      "1": 15,
      "2": 16,
      "3": 17,
      "4": 18,
      "5": 19
    }
  },
  "probing": {
    "Ask for Explanation": {
      "1": 20,
      "2": 21,
      "3": 22,
      "4": 23,
      "5": 24
    },
    "Seek Self-Correction": {
      "1": 25,
      "2": 26,
      "3": 27,
      "4": 28,
      "5": 29
    },
    "Hypothetical Variation": {
      "1": 30,
      "2": 31,
      "3": 32,
      "4": 33,
      "5": 34
    },
    "Check Understanding/Concept": {
      "1": 35,
      "2": 36,
      "3": 37,
      "4": 38,
      "5": 39
    },
    "Encourage Comparison": {
      "1": 40,
      "2": 41,
      "3": 42,
      "4": 43,
      "5": 44
    }
  },
  "telling": {
    "Partial Reveal (Strategy)": {
      "1": 45,
      "2": 46,
      "3": 47,
      "4": 48,
      "5": 49
    },
    "Full Reveal (Answer)": {
      "1": 50,
      "2": 51,
      "3": 52,
      "4": 53,
      "5": 54
    },
    "Corrective Explanation": {
      "1": 55,
      "2": 56,
      "3": 57,
      "4": 58,
      "5": 59
    }
  },
  "generic": {
    "Greeting/Farewell": {
      "1": 60,
      "2": 61,
      "3": 62,
      "4": 63,
      "5": 64
    },
    "Acknowledgment/Praise": {
      "1": 65,
      "2": 66,
      "3": 67,
      "4": 68,
      "5": 69
    },
    "Summarize Progress": {
      "1": 70,
      "2": 71,
      "3": 72,
      "4": 73,
      "5": 74
    },
    "General Inquiry/Filler": {
      "1": 75,
      "2": 76,
      "3": 77,
      "4": 78,
      "5": 79
    }
  }
}

def get_next_action_id(row):
    """
    Maps the action category, subcategory, and hint strength to an action ID.
    """
    try:
        cat = row["next_action_category"].lower()
        sub = row["next_action_subcategory"]
        # Add 1 because CSV hint_strength is 0..4; the dictionary keys are 1..5 as strings
        hint_str = str(int(row["next_action_hint_strength"]) + 1)
        return ACTION_MAP[cat][sub][hint_str]
    except (KeyError, ValueError, TypeError):
        return None

# Apply the function to generate next_action_id
df["next_action_id"] = df.apply(get_next_action_id, axis=1)

# Save with action IDs
df.to_csv("/content/part1_mathdial_complete_with_action_ids.csv", index=False)
print("CSV with action IDs saved to: /content/part1_mathdial_complete_with_action_ids.csv")

Found 64 rows with extraction failures
Fixed: Changed category from 'probing' to 'focus' for subcategory 'seek next step'
Fixed: Changed category from 'telling' to 'generic' for subcategory 'acknowledgment/praise'
Fixed: Changed category from 'telling' to 'generic' for subcategory 'acknowledgment/praise'
Fixed: Changed category from 'telling' to 'generic' for subcategory 'acknowledgment/praise'
Fixed: Changed category from 'telling' to 'generic' for subcategory 'acknowledgment/praise'
Fixed: Changed category from 'telling' to 'generic' for subcategory 'acknowledgment/praise'
Fixed: Changed category from 'telling' to 'generic' for subcategory 'acknowledgment/praise'
Fixed: Changed category from 'focus' to 'probing' for subcategory 'seek self-correction'
Fixed: Changed category from 'telling' to 'generic' for subcategory 'acknowledgment/praise'
Fixed: Changed category from 'probing' to 'focus' for subcategory 'seek next step'
Fixed: Changed category from 'telling' to 'generic' for subcat

In [5]:
df = pd.read_csv("/content/complete.csv")
columns = {
    'extraction_failed',
    'retry_count',
    'error_message'
}

df.drop(columns=columns, inplace=True)
df[:10]

,question,qid,scenario,ground_truth,state,next_action,convo_turn,action_type,done,raw_model_response,listen_to_feedback,problem_progress,correct_solution,next_action_hint_strength,next_action_category,next_action_subcategory,next_action_id
0,Nancy is filling an aquarium for her fish. She...,5000012,1,First calculate the volume of the aquarium by ...,"['', 'START']","Teacher: (probing)Steven, If you had 4 of some...",1,probing,0,"```json\n{\n ""state"": {\n ""listen_to_feedb...",0.0,0.0,0.0,1.0,probing,Ask for Explanation,21.0
1,Nancy is filling an aquarium for her fish. She...,5000012,1,First calculate the volume of the aquarium by ...,"['Teacher: (probing)Steven, If you had 4 of so...",Teacher: (probing)So if Nancy triples the 18 c...,2,probing,0,"{\n ""state"": {\n ""listen_to_feedback"": 1,\...",1.0,25.0,0.0,1.0,probing,Check Understanding/Concept,36.0
2,Nancy is filling an aquarium for her fish. She...,5000012,1,First calculate the volume of the aquarium by ...,['Teacher: (probing)So if Nancy triples the 18...,Teacher: (generic)Exactly correct!,3,generic,1,"{\n ""state"": {\n ""listen_to_feedback"": 1,\...",1.0,100.0,1.0,1.0,generic,Acknowledgment/Praise,66.0
3,James has 20 pairs of red socks and half as ma...,5000982,3,He has 20/2=10 black socks\nSo he has 20+10=30...,"['', 'START']","Teacher: (probing)DeAndre, you successfully an...",1,probing,0,"```json\n{\n ""state"": {\n ""listen_to_feedb...",0.0,0.0,0.0,1.0,probing,Ask for Explanation,21.0
4,James has 20 pairs of red socks and half as ma...,5000982,3,He has 20/2=10 black socks\nSo he has 20+10=30...,"['Teacher: (probing)DeAndre, you successfully ...",Teacher: (probing)could you tell me the total ...,2,probing,0,"{\n ""state"": {\n ""listen_to_feedback"": 1,\...",1.0,75.0,0.0,2.0,probing,Check Understanding/Concept,37.0
5,James has 20 pairs of red socks and half as ma...,5000982,3,He has 20/2=10 black socks\nSo he has 20+10=30...,['Teacher: (probing)could you tell me the tota...,Teacher: (probing)could you tell me the total ...,3,probing,0,"```json\n{\n ""state"": {\n ""listen_to_feedb...",1.0,25.0,0.0,1.0,probing,Check Understanding/Concept,36.0
6,James has 20 pairs of red socks and half as ma...,5000982,3,He has 20/2=10 black socks\nSo he has 20+10=30...,['Teacher: (probing)could you tell me the tota...,Teacher: (probing)What are the total number of...,4,probing,0,"```json\n{\n ""state"": {\n ""listen_to_feedb...",0.0,50.0,0.0,2.0,probing,Check Understanding/Concept,37.0
7,James has 20 pairs of red socks and half as ma...,5000982,3,He has 20/2=10 black socks\nSo he has 20+10=30...,['Teacher: (probing)What are the total number ...,Teacher: (generic)Super!,5,generic,1,"{\n ""state"": {\n ""listen_to_feedback"": 0,\...",0.0,25.0,0.0,1.0,generic,General Inquiry/Filler,76.0
8,Jason drives past 3 convenience stores on his ...,5000274,4,First find the additional distance between the...,"['', 'START']",Teacher: (probing)What is the distance between...,1,probing,0,"```json\n{\n ""state"": {\n ""listen_to_feedb...",0.0,0.0,0.0,1.0,probing,Ask for Explanation,21.0
9,Jason drives past 3 convenience stores on his ...,5000274,4,First find the additional distance between the...,['Teacher: (probing)What is the distance betwe...,Teacher: (probing)What is the distance between...,2,probing,0,"```json\n{\n ""state"": {\n ""listen_to_feedb...",1.0,10.0,0.0,1.0,probing,Check Understanding/Concept,36.0


In [7]:
import pandas as pd
import numpy as np

# Load the DataFrame with action IDs (that currently have floats)
df = pd.read_csv("/content/complete.csv")

# Convert next_action_hint_strength to integer
df["next_action_hint_strength"] = df["next_action_hint_strength"].astype(int)

# Convert next_action_id to integer (when values are not NaN)
df["next_action_id"] = df["next_action_id"].apply(lambda x: int(x) if pd.notna(x) else x)

# Reset index to ensure sequential processing
df.reset_index(drop=True, inplace=True)

# Create progress_delta column
df["progress_delta"] = 0

# Calculate progress_delta using convo_turn to identify conversation breaks
for i in range(1, len(df)):
    # Check if this is not the first turn in a conversation
    if df.loc[i, "convo_turn"] != 1:
        # Calculate the progress difference from the previous turn
        df.loc[i, "progress_delta"] = df.loc[i, "problem_progress"] - df.loc[i-1, "problem_progress"]

# Create previous_action_id column and set initial value to 0
df["previous_action_id"] = 0

# Set previous_action_id for each row
for i in range(1, len(df)):
    # Check if this is not the first turn in a conversation
    if df.loc[i, "convo_turn"] != 1:
        # Set the previous action ID to the next_action_id of the previous row
        df.loc[i, "previous_action_id"] = df.loc[i-1, "next_action_id"]

# Save the final DataFrame to CSV
df.to_csv("/content/final.csv", index=False)

print("Processing complete!")
print(f"Total rows processed: {len(df)}")
print(f"Data saved to: /content/final.csv")

# Display a sample of the DataFrame to verify changes
print("\nSample of processed DataFrame:")
print(df[["next_action_id", "previous_action_id", "next_action_hint_strength", "problem_progress", "progress_delta"]].head(10))

Processing complete!
Total rows processed: 12018
Data saved to: /content/final.csv

Sample of processed DataFrame:
   next_action_id  previous_action_id  next_action_hint_strength  \
0            21.0                 0.0                          1   
1            36.0                21.0                          1   
2            66.0                36.0                          1   
3            21.0                 0.0                          1   
4            37.0                21.0                          2   
5            36.0                37.0                          1   
6            37.0                36.0                          2   
7            76.0                37.0                          1   
8            21.0                 0.0                          1   
9            36.0                21.0                          1   

   problem_progress  progress_delta  
0               0.0               0  
1              25.0              25  
2             100.0   